# mlops-ai library showcase

**Hello user!** 👋

This is an end-to-end notebook to showcase key features of [mlops-ai](https://pypi.org/project/mlops-ai/) library, detailed library documentation can be also found [here](https://mlops-ai.github.io/mlops/library_docs/library_overview.html). <br> [MLOps](https://github.com/mlops-ai/mlops) is an open-source project to help Machine Learning professionals during managing model creation process (**tracking** module), but also monitoring a deployed model working on real-world data (**monitoring** module) with an optional email alerting system (**email alerts** module). More information can be found inside [README](https://github.com/mlops-ai/mlops?tab=readme-ov-file#mlops).

## Library and application installation 

So as to install the library run, the following cells. 

In [1]:
!pip install mlops-ai tqdm

In [2]:
!pip show mlops-ai

Name: mlops-ai
Version: 1.2.8
Summary: Mlops-ai library for managing machine learning projects, experiments, iterations and datasets.
Home-page: 
Author: Kacper Pękalski, Kajetan Szal, Jędrzej Rybczyński
Author-email: kac.pekalski1@gmail.com
License: Apache License 2.0
Location: c:\users\jedryb\anaconda3\lib\site-packages
Requires: json2html, requests, scikit-learn, torch
Required-by: 


To install the application follow the steps in [README](https://github.com/mlops-ai/mlops?tab=readme-ov-file#installation--usage). Essentially, you need to have [docker](https://docs.docker.com/get-docker/) and 
[docker-compose](https://docs.docker.com/compose/install/) installed on your machine. 
Then, you can clone the repository and run following command:

```bash
docker-compose up
```

After that you can access the application at [http://localhost:3000](http://localhost:3000).

## Tracking module

### Creating a project

MLOps project is a single machine learning project placeholder that consists of multiple experiments and models run as iterations.

In [3]:
from mlops.tracking import create_project, get_project, set_active_project

In [4]:
project = create_project(
    title="Iris classification", 
    description="This project is focused on three Iris species multi-classification",
    status='in_progress'
)

In [5]:
set_active_project(project_id=project['_id'])

'Active project set to: 6596acbb7826134dfca22750'

### Creating a first experiment

MLOps experiment is a machine learning experiment that can contain many iterations. Let's create an example experiment about KNN models.

In [6]:
from mlops.tracking import create_experiment, get_experiment, set_active_experiment

In [7]:
experiment = create_experiment(
    # project_id=project['_id'] don't have to pass, since we have set active project
    name="KNN models",
    description="Experimenting with different KNN models"
)

In [8]:
set_active_experiment(experiment_id=experiment['id'])

'Active experiment set to: 6596acbb7826134dfca22751'

### Creating multiple iterations for our experiment

Creating some example iterations of [KNN scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) machine learning models trained on Iris dataset.

In [9]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

url = 'https://raw.githubusercontent.com/TripathiAshutosh/dataset/main/iris.csv'
df = pd.read_csv(filepath_or_buffer=url, sep=',')
y = LabelEncoder().fit_transform(df['class'])
X = df.drop(columns=['class'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify = y, random_state=42)

In [10]:
from mlops.tracking import start_iteration


def log_single_iteration(iteration_name: str, model_name: str,
                         model_params: dict = None,
                         metrics: dict = None,
                         model_path: str = None,
                         dataset_id: str = None,
                         interactive_charts: list = None):
    """
    Util function for creating single mlops iteration.
    
    Args:
        iteration_name (str): name of the whole iteration
        model_name (str): name of the logged model
        model_params (dict): parameters of model
        metrics (dict): model metrics
        model_path (str): path to saved model file
        dataset_id (str): id to dataset from datasets tab
    """
    with start_iteration(iteration_name=iteration_name) as iteration:
        if model_params:
            iteration.log_parameters(parameters=model_params)
            
        if metrics:
            iteration.log_metrics(metrics=metrics)
            
        if model_path:
            iteration.log_path_to_model(path_to_model=model_path)
        
        if dataset_id:
            iteration.log_dataset(dataset_id=dataset_id)

#### 1st iteration

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


parameters = {'n_neighbors': 5, 'metric': 'minkowski', 'algorithm': 'auto'}
model = KNeighborsClassifier(**parameters)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
metrics = {
    'accuracy': round(accuracy_score(y_test, prediction), 3),
    'precision': round(precision_score(y_test, prediction, average='macro'), 3),
    'recall': round(recall_score(y_test, prediction, average='macro'), 3),
    'f1': round(f1_score(y_test, prediction, average='macro'), 3)
}

log_single_iteration(iteration_name='KNN v1',
                     model_name='KNN',
                     model_params=parameters,
                     metrics=metrics)

#### 2nd iteration

In [12]:
import pickle
import os

parameters = {'n_neighbors': 15, 'metric': 'cosine', 'algorithm': 'auto'}
model = KNeighborsClassifier(**parameters)
model.fit(X_train, y_train)
prediction = model.predict(X_test.values)
metrics = {
    'accuracy': round(accuracy_score(y_test, prediction), 3),
    'precision': round(precision_score(y_test, prediction, average='macro'), 3),
    'recall': round(recall_score(y_test, prediction, average='macro'), 3),
    'f1': round(f1_score(y_test, prediction, average='macro'), 3)
}
with open('../test_files/knn_v2.pkl', 'wb') as f:
    pickle.dump(model, f)

log_single_iteration(iteration_name='KNN v2',
                     model_name='KNN',
                     model_params=parameters,
                     metrics=metrics,
                     model_path=os.path.abspath('../test_files/knn_v2.pkl'))

#### 3rd iteration

In [13]:
parameters = {'n_neighbors': 45, 'metric': 'manhattan', 'algorithm': 'ball_tree'}
model = KNeighborsClassifier(**parameters)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
metrics = {
    'accuracy': round(accuracy_score(y_test, prediction), 3),
    'precision': round(precision_score(y_test, prediction, average='macro'), 3),
    'recall': round(recall_score(y_test, prediction, average='macro'), 3),
    'f1': round(f1_score(y_test, prediction, average='macro'), 3)
}

log_single_iteration(iteration_name='KNN v3',
                     model_name='KNN',
                     model_params=parameters,
                     metrics=metrics)

### Creating iterations with dataset

MLOps datasets are stored in a separate tab. They are not actual data, rather than URL or local path attachments to actual data. 

#### Create a dataset from library (from website is better tho) 

In [14]:
from mlops.tracking import create_dataset

dataset_v1 = create_dataset(
    dataset_name="Iris dataset",
    path_to_dataset="https://www.kaggle.com/datasets/uciml/iris",
    dataset_description="Famous Iris species dataset",
    tags="iris,kaggle,classification,multiclass",
    version="1.0"
)

In [15]:
dataset_v2 = create_dataset(
    dataset_name="Iris dataset",
    path_to_dataset="https://www.kaggle.com/datasets/arshid/iris-flower-dataset",
    dataset_description="Famous Iris species dataset",
    tags="iris,kaggle,classification,multiclass",
    version="2.0"
)

#### Create a separable experiment for that

In [16]:
experiment = create_experiment(
    name="Iterations with dataset",
    description="Iterations with dataset showcase"
)

set_active_experiment(experiment_id=experiment['id'])

'Active experiment set to: 6596acbf7826134dfca22757'

#### Log iterations with different datasets

In [17]:
from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators': 100, 'max_depth': 5}
model = RandomForestClassifier(**parameters)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
metrics = {
    'accuracy': round(accuracy_score(y_test, prediction), 3),
    'precision': round(precision_score(y_test, prediction, average='macro'), 3),
    'recall': round(recall_score(y_test, prediction, average='macro'), 3),
    'f1': round(f1_score(y_test, prediction, average='macro'), 3)
}

log_single_iteration(iteration_name='RF with dataset v1',
                     model_name='Random Forest',
                     model_params=parameters,
                     metrics=metrics,
                     dataset_id=dataset_v1['_id'])

In [18]:
parameters = {'n_estimators': 500, 'max_depth': 10}
model = RandomForestClassifier(**parameters)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
metrics = {
    'accuracy': round(accuracy_score(y_test, prediction), 3),
    'precision': round(precision_score(y_test, prediction, average='macro'), 3),
    'recall': round(recall_score(y_test, prediction, average='macro'), 3),
    'f1': round(f1_score(y_test, prediction, average='macro'), 3)
}

log_single_iteration(iteration_name='RF with dataset v2',
                     model_name='Random Forest',
                     model_params=parameters,
                     metrics=metrics,
                     dataset_id=dataset_v2['_id'])

### Creating iterations with Interactive Charts

MLOps allows to create interactive charts from data that can be displayed on site with iterations.

In [19]:
experiment = create_experiment(
    name="Iterations with charts",
    description="Iterations with charts showcase"
)

set_active_experiment(experiment_id=experiment['id'])

'Active experiment set to: 6596acc17826134dfca2275a'

#### Simple pyTorch neural network model

For that purpose, let's create simple pyTorch NN model for Iris classification and based on that visualize learning curves.

In [20]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

BATCH_SIZE=32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [21]:
from torch import nn

class BaselineNN(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape)
        )
    
    def forward(self, x):
        return self.layer_stack(x)

In [22]:
import numpy as np

class MonitoredModelWrapper:
    """
    A wrapper for monitored model that does not have .predict() method from dataframe (like scikit-learn API).
    In this example, we will use a PyTorch model for Iris classification.
    """

    def __init__(self, model: object):
        self.model: object = model

    def predict(self, data: pd.DataFrame) -> np.array:
        """
        Predicts target values from input pandas dataframe.

        Args:
            data (pd.DataFrame): Input data.

        Returns:
            np.array: Predicted target values.
        """
        X_tensor = torch.tensor(data.values, dtype=torch.float32)

        with torch.inference_mode():
            y_logits = self.model(X_tensor)
            y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

        return y_pred.numpy()

#### 1st iteration

In [23]:
torch.manual_seed(42)

HIDDEN_UNITS=10
LEARNING_RATE=0.01

model = BaselineNN(input_shape=X_train.shape[1],
                   hidden_units=HIDDEN_UNITS,
                   output_shape=pd.Series(y).nunique())

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [24]:
from tqdm.auto import tqdm
torch.manual_seed(42)


def train_model(model, epochs: int = 10):
    """
    Util function for training pyTorch model
    
    Args:
        model: torch model instance
        epochs (int): number of epochs
    """
    torch.manual_seed(42)
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    
    ## Training
    for epoch in tqdm(range(epochs), desc="Training", unit="epoch", total=epochs):
        
        train_loss, train_acc = 0, 0
        
        for batch, (X, y) in enumerate(train_loader):
            model.train() 
            y_pred = model(X)
            
            loss = loss_fn(y_pred, y)
            train_loss += loss
            train_acc += accuracy_score(torch.softmax(y_pred, dim=1).argmax(dim=1), y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        train_acc /= len(train_loader)

        ## Validation
        val_loss, val_acc = 0, 0 
        model.eval()
        with torch.inference_mode():
            for X, y in test_loader:
                val_pred = model(X)
                val_loss += loss_fn(val_pred, y)
                val_acc += accuracy_score(torch.softmax(val_pred, dim=1).argmax(dim=1), y)

            val_loss /= len(test_loader)
            val_acc /= len(test_loader)

        print(f"Epoch: {epoch} | Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}% | Val loss: {val_loss:.5f}, Val acc: {val_acc:.2f}%")
        train_losses.append(round(train_loss.item(), 3))
        train_accs.append(round(train_acc.item(), 3))
        val_losses.append(round(val_loss.item(), 3))
        val_accs.append(round(val_acc.item(), 3))
        
    return train_losses, train_accs, val_losses, val_accs


def evaluate_model(model):
    """
    Util function for evaluating pyTorch model, i.e. returning predictions
    
    Args:
        model: torch model instance
    """
    model.eval()
    
    with torch.inference_mode():
        y_logits = model(X_test_tensor)
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
        
    return y_pred.tolist()

In [25]:
EPOCHS=10
train_losses, train_accs, val_losses, val_accs = train_model(model, epochs=EPOCHS)
y_pred = evaluate_model(model)

Training:   0%|          | 0/10 [00:00<?, ?epoch/s]

Epoch: 0 | Train loss: 1.17898 | Train acc: 0.34% | Val loss: 0.99695, Val acc: 0.72%
Epoch: 1 | Train loss: 1.04611 | Train acc: 0.42% | Val loss: 1.00933, Val acc: 0.36%
Epoch: 2 | Train loss: 0.97937 | Train acc: 0.57% | Val loss: 0.94357, Val acc: 0.64%
Epoch: 3 | Train loss: 0.91316 | Train acc: 0.72% | Val loss: 0.86912, Val acc: 0.64%
Epoch: 4 | Train loss: 0.85918 | Train acc: 0.78% | Val loss: 0.80718, Val acc: 0.81%
Epoch: 5 | Train loss: 0.80410 | Train acc: 0.74% | Val loss: 0.75446, Val acc: 0.72%
Epoch: 6 | Train loss: 0.76311 | Train acc: 0.66% | Val loss: 0.70279, Val acc: 0.72%
Epoch: 7 | Train loss: 0.73427 | Train acc: 0.64% | Val loss: 0.65751, Val acc: 0.72%
Epoch: 8 | Train loss: 0.68466 | Train acc: 0.68% | Val loss: 0.62150, Val acc: 0.76%
Epoch: 9 | Train loss: 0.62407 | Train acc: 0.76% | Val loss: 0.59265, Val acc: 0.93%


In [26]:
parameters = {'batch_size': BATCH_SIZE, 'epochs': EPOCHS, 'learning_rate': LEARNING_RATE}
metrics = {
    'train_loss': round(train_losses[-1], 3),
    'train_acc': round(train_accs[-1], 3),
    'val_loss': round(val_losses[-1], 3),
    'val_acc': round(val_accs[-1], 3)
}
torch_monitored = MonitoredModelWrapper(model=model)
with open('../test_files/torch_model.pkl', 'wb') as f:
    pickle.dump(torch_monitored, f)

with start_iteration(iteration_name='NN v1') as iteration:
    iteration.log_parameters(parameters=parameters)
    iteration.log_metrics(metrics=metrics)
    iteration.log_dataset(dataset_id=dataset_v1['_id'])
    iteration.log_path_to_model(os.path.abspath('../test_files/torch_model.pkl'))
    
    iteration.log_chart(
        chart_name="Loss", chart_type="line",
        x_data=[[i for i in range(len(train_losses))]],
        y_data=[train_losses, val_losses],
        y_data_names=['training loss', 'validation loss'],
        x_label="epochs", y_label="Loss", 
        chart_title='Training vs validation loss',
        comparable=True)
    
    iteration.log_chart(
        chart_name="Accuracy", chart_type="line",
        x_data=[[i for i in range(len(train_losses))]],
        y_data=[train_accs, val_accs],
        y_data_names=['training acc', 'validation acc'],
        x_label="epochs", y_label="Loss", 
        chart_title='Training vs validation accuracy',
        comparable=True)

#### 2nd iteration

In [27]:
torch.manual_seed(42)

HIDDEN_UNITS=100
LEARNING_RATE=0.001

model = BaselineNN(input_shape=X_train.shape[1],
                   hidden_units=HIDDEN_UNITS,
                   output_shape=pd.Series(y).nunique())

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [28]:
EPOCHS=10
train_losses, train_accs, val_losses, val_accs = train_model(model, epochs=EPOCHS)
y_pred = evaluate_model(model)

Training:   0%|          | 0/10 [00:00<?, ?epoch/s]

Epoch: 0 | Train loss: 1.13291 | Train acc: 0.35% | Val loss: 1.04898, Val acc: 0.64%
Epoch: 1 | Train loss: 1.01002 | Train acc: 0.66% | Val loss: 0.97090, Val acc: 0.72%
Epoch: 2 | Train loss: 0.95823 | Train acc: 0.70% | Val loss: 0.92601, Val acc: 0.72%
Epoch: 3 | Train loss: 0.96286 | Train acc: 0.64% | Val loss: 0.89495, Val acc: 0.72%
Epoch: 4 | Train loss: 0.93175 | Train acc: 0.64% | Val loss: 0.86333, Val acc: 0.72%
Epoch: 5 | Train loss: 0.88424 | Train acc: 0.66% | Val loss: 0.83263, Val acc: 0.72%
Epoch: 6 | Train loss: 0.85016 | Train acc: 0.66% | Val loss: 0.80585, Val acc: 0.72%
Epoch: 7 | Train loss: 0.82462 | Train acc: 0.64% | Val loss: 0.78180, Val acc: 0.80%
Epoch: 8 | Train loss: 0.79088 | Train acc: 0.91% | Val loss: 0.76002, Val acc: 0.88%
Epoch: 9 | Train loss: 0.75958 | Train acc: 0.91% | Val loss: 0.73725, Val acc: 0.86%


In [29]:
parameters = {'batch_size': BATCH_SIZE, 'epochs': EPOCHS, 'learning_rate': LEARNING_RATE}
metrics = {
    'train_loss': round(train_losses[-1], 3),
    'train_acc': round(train_accs[-1], 3),
    'val_loss': round(val_losses[-1], 3),
    'val_acc': round(val_accs[-1], 3)
}

with start_iteration(iteration_name='NN v2') as iteration:
    iteration.log_parameters(parameters=parameters)
    iteration.log_metrics(metrics=metrics)
    iteration.log_dataset(dataset_id=dataset_v1['_id'])
    
    iteration.log_chart(
        chart_name="Loss", chart_type="line",
        x_data=[[i for i in range(len(train_losses))]],
        y_data=[train_losses, val_losses],
        y_data_names=['training loss', 'validation loss'],
        x_label="epochs", y_label="Loss", 
        chart_title='Training vs validation loss',
        comparable=True)
    
    iteration.log_chart(
        chart_name="Accuracy", chart_type="line",
        x_data=[[i for i in range(len(train_losses))]],
        y_data=[train_accs, val_accs],
        y_data_names=['training acc', 'validation acc'],
        x_label="epochs", y_label="Loss", 
        chart_title='Training vs validation accuracy',
        comparable=True)

### Creating iterations with Image Charts

MLOps alows not only to log interactive charts, but also presaved charts from images (i.e. from matplotlib or seaborn library).

**<font color='red'>Attention!</font>**<br>
**Unfortunately, this feature does not work on Linux/macOS system.**

In [30]:
experiment = create_experiment(
    name="Iterations with image charts",
    description="Iterations with image charts showcase"
)

set_active_experiment(experiment_id=experiment['id'])

'Active experiment set to: 6596acc47826134dfca22761'

#### Example image charts iterations

In [31]:
with start_iteration("Iteration with image v1") as iteration:
    iteration.log_parameter("n_estimators", 150)
    iteration.log_metric("RMSE", 0.9)
    iteration.log_metric("MAE", 0.45)
    iteration.log_image_chart(name="Image chart 1", 
                              image_path='../test_files/plot-1.png')
    iteration.log_image_chart(name="Image chart 2", 
                              image_path='../test_files/plot-2.png')

In [32]:
with start_iteration("Iteration with image v2") as iteration:
    iteration.log_parameter("n_estimators", 225)
    iteration.log_metric("RMSE", 0.8)
    iteration.log_metric("MAE", 0.41)
    iteration.log_image_chart(name="Image chart 3", 
                              image_path='../test_files/plot-3.png')
    iteration.log_image_chart(name="Image chart 4", 
                              image_path='../test_files/plot-4.png')

## Monitoring module

### Creating a monitored model

Creating a monitored model takes place from the application side. You can select a model from the grid (one that have actual `Model path`, i.e. one of the KNN or NN) and select **Create model** button. Please, name it **Showcase model**.

### Send predictions to the monitored model

When the monitored model is ready, you can send prediction requests to the model with pd.DataFrame as an input data. <br> Prediction is made as a batch prediction for the whole DataFrame, below you can see comparison between single batch prediction for the whole dataframe vs many instance predictions for each sample from DataFrame.

#### Batch prediction

In [33]:
from mlops.monitoring import send_prediction
import time

start_time = time.time()

_ = send_prediction(
    model_name='Showcase model',
    data=X_test
)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 0.02651524543762207 seconds


#### Instance predictions

In [34]:
start_time = time.time()

for idx, val in X_test.iterrows():
    _ = send_prediction(
        model_name='Showcase model',
        data=pd.DataFrame([val])
    )

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 2.232327699661255 seconds


### Get predictions from the monitored model

You can also get predictions from the monitored model afterwards, i.e. after annotating actual value from the application.

In [35]:
from mlops.monitoring import get_model_by_name

pd.DataFrame(get_model_by_name(model_name='Showcase model')['predictions_data'])

,id,prediction_date,input_data,prediction,actual
0,6596acd97826134dfca22769,2024-01-04T14:04:25.851000,"{'sepal-length': 5.1, 'sepal-width': 3.7, 'pet...",0.0,None
1,6596acd97826134dfca2276a,2024-01-04T14:04:25.851000,"{'sepal-length': 5.0, 'sepal-width': 3.4, 'pet...",0.0,None
2,6596acd97826134dfca2276b,2024-01-04T14:04:25.851000,"{'sepal-length': 5.0, 'sepal-width': 3.2, 'pet...",0.0,None
3,6596acd97826134dfca2276c,2024-01-04T14:04:25.851000,"{'sepal-length': 5.1, 'sepal-width': 3.3, 'pet...",0.0,None
4,6596acd97826134dfca2276d,2024-01-04T14:04:25.851000,"{'sepal-length': 6.4, 'sepal-width': 2.8, 'pet...",2.0,None
...,...,...,...,...,...
145,6596acdf7826134dfca227fa,2024-01-04T14:04:31.347000,"{'sepal-length': 5.5, 'sepal-width': 2.3, 'pet...",1.0,None
146,6596acdf7826134dfca227fb,2024-01-04T14:04:31.382000,"{'sepal-length': 4.6, 'sepal-width': 3.6, 'pet...",0.0,None
147,6596acdf7826134dfca227fc,2024-01-04T14:04:31.417000,"{'sepal-length': 7.1, 'sepal-width': 3.0, 'pet...",2.0,None
148,6596acdf7826134dfca227fd,2024-01-04T14:04:31.452000,"{'sepal-length': 6.3, 'sepal-width': 3.4, 'pet...",2.0,None


## Email alerts module

### Setup [MailGun](https://www.mailgun.com/) variables

MLOps email alerts module is integrated with [MailGun](https://www.mailgun.com/). So as to receive email alerts after iteration or sending a prediction, you have to register at MailGun and provide your data as below.

In [36]:
from mlops.config.config import settings

settings.set_mailgun_domain('your-mailgun-domain')
settings.set_mailgun_api_key('your-mailgun-api-key')
settings.set_user_email('your-email') 
settings.set_send_emails_flag(False)